## 初始数据

In [115]:
!pip install lightgbm sklearn pandas

In [116]:
import pandas as pd
import numpy as np

In [117]:
train_final = pd.read_csv('./sample_data/train_final.csv', engine='python')
test_final = pd.read_csv('./sample_data/test_final.csv', engine='python')

In [118]:
train_final.shape

(50000, 146)

In [119]:
test_final.shape

(50000, 146)

In [120]:
train_final.columns.to_list()

['continuous_annual_inc',
 'continuous_annual_inc_joint',
 'continuous_delinq_2yrs',
 'continuous_dti',
 'continuous_dti_joint',
 'continuous_fico_range_high',
 'continuous_fico_range_low',
 'continuous_funded_amnt',
 'continuous_funded_amnt_inv',
 'continuous_inq_last_6mths',
 'continuous_installment',
 'continuous_int_rate',
 'continuous_last_fico_range_high',
 'continuous_last_fico_range_low',
 'continuous_loan_amnt',
 'loan_status',
 'continuous_mths_since_last_delinq',
 'continuous_mths_since_last_major_derog',
 'continuous_mths_since_last_record',
 'continuous_open_acc',
 'continuous_pub_rec',
 'discrete_addr_state_1_one_hot',
 'discrete_addr_state_2_one_hot',
 'discrete_addr_state_3_one_hot',
 'discrete_addr_state_4_one_hot',
 'discrete_addr_state_5_one_hot',
 'discrete_addr_state_6_one_hot',
 'discrete_addr_state_7_one_hot',
 'discrete_addr_state_8_one_hot',
 'discrete_addr_state_9_one_hot',
 'discrete_addr_state_10_one_hot',
 'discrete_addr_state_11_one_hot',
 'discrete_addr_s

In [121]:
train_final['loan_status'].unique()

array([1, 0])

In [122]:
test_final['loan_status'].unique()

array([1, 0])

## 训练与测试数据

按照列的相关含义选取了一下数据。

In [123]:
train_dt = train_final.loc[:, test_final.columns.isin(['continuous_annual_inc',
 'continuous_delinq_2yrs',
 'continuous_dti',
 'continuous_fico_range_high',
 'continuous_fico_range_low',
 'continuous_funded_amnt', 
 'continuous_inq_last_6mths',
 'continuous_installment',
 'continuous_int_rate',
 'continuous_last_fico_range_high',
 'continuous_last_fico_range_low',
 'continuous_loan_amnt',
 'loan_status', 
 'continuous_open_acc',
 'continuous_pub_rec', 
 'discrete_grade_1_one_hot',
 'discrete_grade_2_one_hot',
 'discrete_grade_3_one_hot',
 'discrete_grade_4_one_hot',
 'discrete_grade_5_one_hot',
 'discrete_grade_6_one_hot',
 'discrete_grade_7_one_hot',
 'discrete_home_ownership_1_one_hot',
 'discrete_home_ownership_2_one_hot',
 'discrete_home_ownership_3_one_hot',
 'discrete_home_ownership_4_one_hot',
 'discrete_term_1_one_hot',
 'discrete_term_2_one_hot'])]

train_dt.shape

(50000, 28)

In [124]:
# test_new = test_final.loc[:,~test_final.columns.isin(["continuous_annual_inc","one_hot"])]
test_dt = test_final.loc[:, test_final.columns.isin(['continuous_annual_inc',
 'continuous_delinq_2yrs',
 'continuous_dti',
 'continuous_fico_range_high',
 'continuous_fico_range_low',
 'continuous_funded_amnt', 
 'continuous_inq_last_6mths',
 'continuous_installment',
 'continuous_int_rate',
 'continuous_last_fico_range_high',
 'continuous_last_fico_range_low',
 'continuous_loan_amnt',
 'loan_status',
 'continuous_open_acc',
 'continuous_pub_rec', 
 'discrete_grade_1_one_hot',
 'discrete_grade_2_one_hot',
 'discrete_grade_3_one_hot',
 'discrete_grade_4_one_hot',
 'discrete_grade_5_one_hot',
 'discrete_grade_6_one_hot',
 'discrete_grade_7_one_hot',
 'discrete_home_ownership_1_one_hot',
 'discrete_home_ownership_2_one_hot',
 'discrete_home_ownership_3_one_hot',
 'discrete_home_ownership_4_one_hot', 
 'discrete_term_1_one_hot',
 'discrete_term_2_one_hot'])]

test_dt.shape

(50000, 28)

## 相关代码

In [125]:
import io
import lightgbm as lgb
import multiprocessing
from copy import deepcopy
from contextlib import redirect_stdout
from dataclasses import asdict, dataclass, field, fields, is_dataclass
from hyperopt import fmin, tpe, hp, pyll, space_eval
from hyperopt.pyll.base import scope
import hyperopt.pyll.stochastic
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score


In [126]:
cpu_count = 4
use_gpu = False

In [177]:
space= {
    'num_threads': hp.choice('num_threads', [cpu_count]),
    # 'num_leaves': hp.choice('num_leaves', [64]),  
    'num_leaves': scope.int(hp.quniform('epochs', 61, 68, 1)),
    'metric': hp.choice('metric', ['binary_error']),
    'num_round': hp.choice('num_round', [2000]),
    'objective': hp.choice('objective', ['binary']),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
    'feature_fraction': hp.uniform('feature_fraction', 0.5, 1.0),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.8, 1.0),
    'device_type': hp.choice('device_tpye', ['gpu']) if use_gpu else hp.choice('device_type', ['cpu']),
    'boosting': hp.choice('boosting', ['gbdt', 'dart', 'goss']),
    'extra_trees': hp.choice('extra_tress', [False, True]),
    'drop_rate': hp.uniform('drop_rate', 0, 0.2),
    'uniform_drop': hp.choice('uniform_drop', [True, False]),
    'lambda_l1': hp.uniform('lambda_l1', 0, 10),
    'lambda_l2': hp.uniform('lambda_l2', 0, 10),
    'min_gain_to_split': hp.uniform('min_gain_to_split', 0, 1), 
    'min_data_in_bin': hp.choice('min_data_in_bin', [3, 5, 10, 15, 20, 50])
}

print(hyperopt.pyll.stochastic.sample(space))

{'bagging_fraction': 0.9543293468399341, 'boosting': 'dart', 'device_type': 'cpu', 'drop_rate': 0.18651111973475387, 'extra_trees': False, 'feature_fraction': 0.5687545083289387, 'lambda_l1': 2.4089984552453947, 'lambda_l2': 8.296832482178472, 'learning_rate': 0.05740160202484405, 'metric': 'binary_error', 'min_data_in_bin': 5, 'min_gain_to_split': 0.6897606551041301, 'num_leaves': 67, 'num_round': 2000, 'num_threads': 4, 'objective': 'binary', 'uniform_drop': True}


In [128]:
class FitterBase(object):
    def __init__(self, label, metric, max_eval=100, opt=None):
        self.label = label
        self.metric = metric
        self.opt_params = dict()
        self.max_eval = max_eval
        self.opt = opt

    def get_loss(self, y, y_pred):
        if self.metric == 'error':
            return 1 - accuracy_score(y, y_pred)
        elif self.metric == 'precision':
            return 1 - precision_score(y, y_pred)
        elif self.metric == 'recall':
            return 1 - recall_score(y, y_pred)
        elif self.metric == 'macro_f1':
            return 1 - f1_score(y, y_pred, average='macro')
        elif self.metric == 'micro_f1':
            return 1 - f1_score(y, y_pred, average='micro')
        elif self.metric == 'auc':  # TODO: Add a warning checking if y_predict is all [0, 1], it should be probability
            return 1 - roc_auc_score(y, y_pred)
        else:
            raise Exception("Not implemented yet.")

In [183]:
class LGBFitter(FitterBase):
    def __init__(self, label='label', metric='error', opt=None, max_eval=100):
        super(LGBFitter, self).__init__(label, metric, max_eval)
        if opt is not None:
            self.opt = opt
        else:
            self.opt = space
        self.best_round = None
        self.clf = None

    def train(self, train_df, eval_df, params=None, use_best_eval=True):
        self.best_round = None
        dtrain = lgb.Dataset(train_df.drop(columns=[self.label]), train_df[self.label])
        deval = lgb.Dataset(eval_df.drop(columns=[self.label]), eval_df[self.label])
        evallist = [dtrain, deval]
        if params is None:
            use_params = deepcopy(self.opt_params)            
        else:
            use_params = deepcopy(params)

        num_round = use_params.pop('num_round')
        if use_best_eval:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, num_round, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            min_error = np.inf
            min_index = 0
            for idx in range(len(output) - 1):
                if len(output[idx].split("\t")) == 3:
                    temp = float(output[idx].split("\t")[2].split(":")[1])
                    if min_error > temp:
                        min_error = temp
                        min_index = int(output[idx].split("\t")[0][1:-1])
            print("The minimum is attained in round %d" % (min_index + 1))
            self.best_round = min_index + 1
            return output
        else:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, num_round, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            self.best_round = num_round
            return output   

    def search_k_fold(self, k_fold, data, use_best_eval=True):
        self.opt_params = dict()

        def train_impl_nfold(params):
            loss = list()
            for train_id, eval_id in k_fold.split(data):
                train_df = data.loc[train_id]
                eval_df = data.loc[eval_id]
                self.train(train_df, eval_df, params, use_best_eval)
                if self.metric == 'auc':
                    y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
                else:
                    y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                               num_iteration=self.best_round) > 0.5).astype(int)
                loss.append(self.get_loss(eval_df[self.label], y_pred))
            return np.mean(loss)

        self.opt_params = fmin(train_impl_nfold, self.opt, algo=tpe.suggest, max_evals=self.max_eval)
    
    def train_k_fold(self, k_fold, train_data, test_data, params=None, drop_test_y=True, use_best_eval=True):
        acc_result = list()
        tpe_result = list()        
        train_pred = np.empty(train_data.shape[0])
        test_pred = np.empty(test_data.shape[0])
        if drop_test_y:
            dtest = test_data.drop(columns=self.label)
        else:
            dtest = test_data

        models = list()
        for train_id, eval_id in k_fold.split(train_data):
            train_df = train_data.loc[train_id]
            eval_df = train_data.loc[eval_id]
            self.train(train_df, eval_df, params, use_best_eval)
            models.append(deepcopy(self.clf))
            train_pred[eval_id] = self.clf.predict(eval_df.drop(columns=self.label), num_iteration=self.best_round)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            acc_result.append(self.get_loss(eval_df[self.label], y_pred))
            tpe_f = self.clf.predict(dtest, num_iteration=self.best_round)
            tpe_result.append(accuracy_score(test_data['loan_status'], tpe_f.round()))            
            test_pred += tpe_f
        test_pred /= k_fold.n_splits 
            
        return train_pred, test_pred, acc_result,tpe_result,models 

## 训练模型

In [130]:
fitter = LGBFitter(label='loan_status')

In [131]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5)

In [132]:
rs = fitter.search_k_fold(kfold, train_dt)

The minimum is attained in round 44
The minimum is attained in round 56
The minimum is attained in round 41
The minimum is attained in round 387
The minimum is attained in round 290
The minimum is attained in round 62
The minimum is attained in round 191
The minimum is attained in round 77
The minimum is attained in round 313
The minimum is attained in round 50
The minimum is attained in round 201
The minimum is attained in round 297
The minimum is attained in round 192
The minimum is attained in round 550
The minimum is attained in round 570
The minimum is attained in round 41
The minimum is attained in round 35
The minimum is attained in round 19
The minimum is attained in round 115
The minimum is attained in round 70
The minimum is attained in round 178
The minimum is attained in round 108
The minimum is attained in round 105
The minimum is attained in round 762
The minimum is attained in round 900
The minimum is attained in round 136
The minimum is attained in round 190
The minimum

In [133]:
best_params = space_eval(fitter.opt, fitter.opt_params)
print(best_params)

{'bagging_fraction': 0.844395861681256, 'boosting': 'goss', 'device_type': 'cpu', 'drop_rate': 0.10552173885412995, 'extra_trees': False, 'feature_fraction': 0.5689392403879664, 'lambda_l1': 6.646990406646976, 'lambda_l2': 3.8132497824007823, 'learning_rate': 0.048709003827455924, 'metric': 'binary_error', 'min_data_in_bin': 5, 'min_gain_to_split': 0.3627069180816052, 'num_leaves': 61, 'num_round': 2000, 'num_threads': 4, 'objective': 'binary', 'uniform_drop': True}


In [136]:
fitter.train_k_fold(kfold, train_dt, test_dt,params=best_params)

The minimum is attained in round 116
The minimum is attained in round 100
The minimum is attained in round 38
The minimum is attained in round 491
The minimum is attained in round 269


(array([0.54717475, 0.98385052, 0.98290955, ..., 0.99323902, 0.99788556,
        0.99651679]),
 array([0.93797567, 0.98912224, 0.96995964, ..., 0.96787571, 0.98809948,
        0.98684264]),
 [0.07210000000000005,
  0.08030000000000004,
  0.08340000000000003,
  0.08420000000000005,
  0.08009999999999995],
 [0.91756, 0.91682, 0.91618, 0.91788, 0.91714],
  <lightgbm.basic.Booster at 0x7fee4df2b6d0>])

上面数组有五个元素，第二个为验证集的损失评分，第三个元素为模型分别对测试集的评分，下同。

## 衍生变量１

变量inc_install　＝　客户年报收入（continuous_annual_inc）/ 12 - 贷款后每月应还款项（continuous_installment）

In [137]:
train_dn = deepcopy(train_dt)
test_dn = deepcopy(test_dt)

In [138]:
train_dn['inc_install'] = (train_dn['continuous_annual_inc'] /12 - train_dn['continuous_installment']) 
test_dn['inc_install'] = (test_dn['continuous_annual_inc'] /12 - test_dn['continuous_installment']) 

In [139]:
train_dn.iloc[0:9, :]

,continuous_annual_inc,continuous_delinq_2yrs,continuous_dti,continuous_fico_range_high,continuous_fico_range_low,continuous_funded_amnt,continuous_inq_last_6mths,continuous_installment,continuous_int_rate,continuous_last_fico_range_high,continuous_last_fico_range_low,continuous_loan_amnt,loan_status,continuous_open_acc,continuous_pub_rec,discrete_grade_1_one_hot,discrete_grade_2_one_hot,discrete_grade_3_one_hot,discrete_grade_4_one_hot,discrete_grade_5_one_hot,discrete_grade_6_one_hot,discrete_grade_7_one_hot,discrete_home_ownership_1_one_hot,discrete_home_ownership_2_one_hot,discrete_home_ownership_3_one_hot,discrete_home_ownership_4_one_hot,discrete_term_1_one_hot,discrete_term_2_one_hot,inc_install
0,55000.0,0.0,5.91,679.0,675.0,3600.0,1.0,123.03,13.99,564.0,560.0,3600.0,1,7.0,0.0,1,0,0,0,0,0,0,1,0,0,0,1,0,4460.303333
1,65000.0,1.0,16.06,719.0,715.0,24700.0,4.0,820.28,11.99,699.0,695.0,24700.0,1,22.0,0.0,1,0,0,0,0,0,0,1,0,0,0,1,0,4596.386667
2,63000.0,0.0,10.78,699.0,695.0,20000.0,0.0,432.66,10.78,704.0,700.0,20000.0,1,6.0,0.0,0,1,0,0,0,0,0,1,0,0,0,0,1,4817.340000
3,104433.0,1.0,25.37,699.0,695.0,10400.0,3.0,289.91,22.45,704.0,700.0,10400.0,1,12.0,0.0,0,0,1,0,0,0,0,1,0,0,0,0,1,8412.840000
4,34000.0,0.0,10.20,694.0,690.0,11950.0,0.0,405.18,13.44,759.0,755.0,11950.0,1,5.0,0.0,1,0,0,0,0,0,0,0,1,0,0,1,0,2428.153333
5,180000.0,0.0,14.67,684.0,680.0,20000.0,0.0,637.58,9.17,654.0,650.0,20000.0,1,12.0,0.0,0,1,0,0,0,0,0,1,0,0,0,1,0,14362.420000
6,85000.0,1.0,17.61,709.0,705.0,20000.0,0.0,631.26,8.49,674.0,670.0,20000.0,1,8.0,0.0,0,1,0,0,0,0,0,1,0,0,0,1,0,6452.073333
7,85000.0,0.0,13.07,689.0,685.0,10000.0,1.0,306.45,6.49,719.0,715.0,10000.0,1,14.0,1.0,0,0,0,1,0,0,0,0,1,0,0,1,0,6776.883333
8,42000.0,0.0,34.80,704.0,700.0,8000.0,0.0,263.74,11.48,679.0,675.0,8000.0,1,8.0,0.0,0,1,0,0,0,0,0,1,0,0,0,1,0,3236.260000


In [188]:
fitter = LGBFitter(label='loan_status')

这里沿用了上面的参数，重新搜索或许效果好一些。

In [147]:
fitter.train_k_fold(kfold, train_dn, test_dn,params=best_params)

The minimum is attained in round 149
The minimum is attained in round 84
The minimum is attained in round 41
The minimum is attained in round 168
The minimum is attained in round 282


(array([0.55903971, 0.98527479, 0.9763533 , ..., 0.99504704, 0.99860456,
        0.99630393]),
 array([0.93551458, 0.99013775, 0.96991113, ..., 0.96704403, 0.98954479,
        0.98801184]),
 [0.07220000000000004,
  0.0807,
  0.08350000000000002,
  0.08579999999999999,
  0.08020000000000005],
 [0.91724, 0.91774, 0.91674, 0.91826, 0.91636],
  <lightgbm.basic.Booster at 0x7fee3c73bd10>])

## 衍生变量２

测试fico评分变化的影响。取当前fico上下届的均值减去上次fico上下届的均值的差，根据该差值是否为负值进行one-hot编码

In [148]:
train_dn2 = deepcopy(train_dt)
test_dn2 = deepcopy(test_dt)

In [149]:

train_dn2['fico_mean_now'] = (train_dn2['continuous_fico_range_high'] + train_dn2['continuous_fico_range_low']) / 2
train_dn2['fico_mean_last'] = (train_dn2['continuous_last_fico_range_high'] + train_dn2['continuous_last_fico_range_low']) / 2
train_dn2['fico_change'] = (train_dn2['fico_mean_now'] - train_dn2['fico_mean_last']) 
train_dn2['fico_change0'] = train_dn2['fico_change'].map(lambda x: 0 if x >= 0 else 1)
train_dn2['fico_change1'] = train_dn2['fico_change'].map(lambda x: 0 if x < 0 else 1)

In [151]:


test_dn2['fico_mean_now'] = (test_dn2['continuous_fico_range_high'] + test_dn2['continuous_fico_range_low']) / 2
test_dn2['fico_mean_last'] = (test_dn2['continuous_last_fico_range_high'] + test_dn2['continuous_last_fico_range_low']) / 2
test_dn2['fico_change'] = (test_dn2['fico_mean_now'] - test_dn2['fico_mean_last']) 
test_dn2['fico_change0'] = test_dn2['fico_change'].map(lambda x: 0 if x >= 0 else 1)
test_dn2['fico_change1'] = test_dn2['fico_change'].map(lambda x: 0 if x < 0 else 1)


In [152]:
test_dn2.drop(columns=['fico_mean_last','fico_mean_now','fico_change'],inplace=True)
test_dn2.shape

(50000, 30)

In [153]:
train_dn2.drop(columns=['fico_mean_last','fico_mean_now','fico_change'],inplace=True)
train_dn2.shape

(50000, 30)

In [155]:
fitter.train_k_fold(kfold, train_dn2, test_dn2,params=best_params)

The minimum is attained in round 79
The minimum is attained in round 131
The minimum is attained in round 36
The minimum is attained in round 81
The minimum is attained in round 296


(array([0.53766253, 0.97101945, 0.98480423, ..., 0.99584519, 0.9988175 ,
        0.99509399]),
 array([0.92360811, 0.98782737, 0.96741355, ..., 0.95963312, 0.98501741,
        0.98300186]),
 [0.07210000000000005,
  0.08089999999999997,
  0.08379999999999999,
  0.08520000000000005,
  0.08040000000000003],
 [0.91716, 0.91738, 0.9162, 0.91756, 0.91722],
  <lightgbm.basic.Booster at 0x7fee3964ef50>])

## 衍生变量３

使用“借款人上次拖欠债务以来的月数（continuous_mths_since_last_delinq）”来进行one-hot编码。

In [156]:
train_dn6 = deepcopy(train_dt)
test_dn6 = deepcopy(test_dt)

In [158]:
df = deepcopy(train_final['continuous_mths_since_last_delinq'])
df.describe()

count    26083.000000
mean        34.197485
std         21.828204
min          0.000000
25%         16.000000
50%         31.000000
75%         50.000000
max        152.000000
Name: continuous_mths_since_last_delinq, dtype: float64

In [159]:
dt = deepcopy(test_final['continuous_mths_since_last_delinq'])
dt.describe()

count    25927.000000
mean        34.184055
std         22.136024
min          0.000000
25%         15.000000
50%         31.000000
75%         50.000000
max        157.000000
Name: continuous_mths_since_last_delinq, dtype: float64

In [160]:
df1 = df.map(lambda x: 1 if pd.isnull(x) else 0)
df2 = df.map(lambda x: 1 if x >= 0 and x <= 50 else 0)
df3 = df.map(lambda x: 1 if x > 50 else 0)

train_dn6['df1'] = df1
train_dn6['df2'] = df2
train_dn6['df3'] = df3


dt1 = dt.map(lambda x: 1 if pd.isnull(x) else 0)
dt2 = dt.map(lambda x: 1 if x >= 0 and x <= 50 else 0)
dt3 = dt.map(lambda x: 1 if x > 50 else 0)

test_dn6['dt1'] = dt1
test_dn6['dt2'] = dt2
test_dn6['dt3'] = dt3

In [190]:
fitter.train_k_fold(kfold, train_dn6, test_dn6,params=best_params)

The minimum is attained in round 117
The minimum is attained in round 550
The minimum is attained in round 29
The minimum is attained in round 436
The minimum is attained in round 110


(array([0.55627501, 0.98744207, 0.9811014 , ..., 0.98852216, 0.99256049,
        0.99389643]),
 array([0.92414882, 0.9862343 , 0.97012028, ..., 0.96241629, 0.98574245,
        0.98502855]),
 [0.07299999999999995,
  0.08109999999999995,
  0.0837,
  0.0837,
  0.08030000000000004],
 [0.91748, 0.9164, 0.91514, 0.91788, 0.91778],
  <lightgbm.basic.Booster at 0x7fee387711d0>])